# Импорт библиотек и создание функций

In [515]:
import numpy as np
import pandas as pd

import re
import sys
import itertools
import datetime
from tqdm.notebook import tqdm
import pandas_profiling

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer

from catboost import CatBoostRegressor
import xgboost as xgb

from hyperopt import tpe, hp, fmin, STATUS_OK, Trials
from hyperopt.pyll.base import scope

import warnings
warnings.filterwarnings("ignore")

In [516]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
Numpy        : 1.20.0


In [517]:
# зафиксируем версии пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [518]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы:
RANDOM_SEED = 42

### Функции предобработки

In [519]:
# заполнение engineDisplacement числовыми значениями
def mape(y_true: np.ndarray,
         y_pred: np.ndarray):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# заполнение owners числовыми значениями
def transf_engineDisplacement_to_float(row: str):
    extracted_value = re.findall('\d\.\d', str(row))
    if extracted_value:
        return float(extracted_value[0])
    return None

# заполнение vehicleTransmission числовыми значениями
def transf_owners_to_float(value: str):
    if isinstance(value, str):
        return float(value.replace('\xa0', ' ').split()[0])
    return value

# заполнение enginePower числовыми значениями
def transf_vehicleTransmission_to_categ(value: str):
    if isinstance(value, str):
        if value in ['MECHANICAL', 'механическая']:
            return 'mechanical'
        else:
            return 'automatic'
    return value



def transf_enginePower_to_float(value: str):
    if isinstance(value, str):
        if value == 'undefined N12':
            return None
        else:
            return float(value.replace(' N12', ''))
    return value


def vis_num_feature(data: pd.DataFrame,
                    column: str,
                    target_column: str,
                    query_for_slicing: str):
    # построение графиков для численных переменных
    plt.style.use('seaborn-paper')
    fig, ax = plt.subplots(2, 3, figsize=(15, 9))
    data[column].plot.hist(ax=ax[0][0])
    ax[0][0].set_title(column)
    sns.boxplot(data=data, y=column, ax=ax[0][1], orient='v')
    sns.scatterplot(data=data.query(query_for_slicing),
                    x=column, y=target_column, ax=ax[0][2])
    np.log2(data[column] + 1).plot.hist(ax=ax[1][0])
    ax[1][0].set_title(f'log2 transformed {column}')
    sns.boxplot(y=np.log2(data[column]), ax=ax[1][1], orient='v')
    plt.show()


def calculate_stat_outliers(data_initial: pd.DataFrame,
                            column: str,
                            log: bool = False):

    data = data_initial.copy()
    if log:
        data[column] = np.log2(data[column] + 1)
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    IQR = q3 - q1
    mask25 = q1 - IQR * 1.5
    mask75 = q3 + IQR * 1.5

    values = {}
    values['borders'] = mask25, mask75
    values['# outliers'] = data[(
        data[column] < mask25)].shape[0], data[data[column] > mask75].shape[0]
    return pd.DataFrame.from_dict(data=values, orient='index', columns=['left', 'right'])


def show_boxplot(data: pd.DataFrame,
                 column: str,
                 target_column: str):
    # построение боксплотов для численных признаков
    fig, ax = plt.subplots(figsize=(14, 4))
    sns.boxplot(x=column, y=target_column,
                data=data.loc[data.loc[:, column].isin(
                    data.loc[:, column].value_counts().index)],
                ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

# DATA

### Парсинг

In [520]:
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# import time
# import requests as r
# import wget
# import gzip
# import xml.etree.ElementTree as ET

# # Читаем файл robots.txt с сайта https://auto.ru/robots.txt и получаем значение параметра Sitemap
# url = "https://auto.ru/robots.txt"
# response = r.get(url)
# url = response.text[response.text.find("Sitemap"):][9:response.text[3625:].find("\n")]
# response = r.get(url)

# # Парсим полученный xml
# root = ET.fromstring(response.text)

# # Добаваляем архивы предложений автомобилей со строками "offers_cars" в массив
# look_for_string = "offers_cars"
# offers_arc = []

# for element in root:
#     if look_for_string in element[1].text: offers_arc.append(element[1].text)

# # Скачиваем архивы с объявлениями
# for arc in offers_arc:
#     wget.download(arc)

# # Создадим список ссылок на объявления
# pages_url_list=[]

# # Открываем и читаем архивы
# for arc in offers_arc:
#     file_name = arc[arc.find("sitemap_offers_cars"):]

#     with gzip.open(file_name, 'rb') as f:
#         file_content = f.read()

#     # Файл внутри представляет собой XML. парсим его
#     root = ET.fromstring(file_content)

#     # Ищем строки, содержашие used - авто с пробегом и добавляем их в массив ссылок
#     look_for_string = "used"
#     for element in root:
#         if look_for_string in element[0].text: pages_url_list.append(element[0].text)

# # Определяем итоговый массив данных
# cars = []

# # Запускаем Selenium
# chrome_options = Options()
# driver = webdriver.Chrome(executable_path=r"chromedriver.exe",options=chrome_options)

# # Открываем каждую ссылку, заносим данные в массив
# for url in pages_url_list:
#     driver.get(url)
#     #Определяем бренд и модель
#     elements = driver.find_elements(By.CLASS_NAME,'CardBreadcrumbs__itemText')
#     brand = elements[3].text
#     model_name = elements[4].text

#     # Определяем кузов
#     element =  driver.find_elements(By.CLASS_NAME,'CardInfoRow_bodytype')
#     bodyType = element[0].text.replace("\n"," ")

#     # Определяем цвет
#     element =  driver.find_elements(By.CLASS_NAME,'CardInfoRow_color')
#     color = element[0].text.replace("\n"," ").replace("Цвет ","")

#     # Определяем параметры двигателя
#     element =  driver.find_elements(By.CLASS_NAME,'CardInfoRow_engine')
#     engine = element[0].text.replace("Двигатель","").split("/")

#     # Получаем значение лошадиных сил, мощность и тип топлива
#     engineDisplacement = engine[0][1:].replace("л ","LTR")
#     enginePower = engine[1][1:].replace("л.с. ", "N12")
#     fuelType = engine[2]

#     # Определяем пробег
#     element =  driver.find_elements(By.CLASS_NAME,'CardInfoRow_kmAge')
#     mileage = element[0].text.replace("Пробег\n","").replace(" км","").replace(" ","")

#     # Определяем год выпуска
#     element =  driver.find_elements(By.CLASS_NAME,'CardInfoRow_year')
#     productionDate = element[0].text.split("\n")[1]

#     # Определяем трансмиссию
#     element = driver.find_elements(By.CLASS_NAME,'CardInfoRow_transmission')
#     vehicleTransmission = element[0].text.split("\n")[1]

#     # Определяем владельца
#     element = driver.find_elements(By.CLASS_NAME,'CardInfoRow_ownersCount')
#     owners = element[0].text.split("\n")[1]

#     # Определяем ПТС
#     element = driver.find_elements(By.CLASS_NAME,'CardInfoRow_pts')
#     pts = element[0].text.split("\n")[1]

#     # Определяем привод
#     element = driver.find_elements(By.CLASS_NAME,'CardInfoRow_drive')
#     wd = element[0].text.split("\n")[1]

#     # Определяем руль
#     element = driver.find_elements(By.CLASS_NAME,'CardInfoRow_wheel')
#     weel = element[0].text.split("\n")[1]

#     # Определяем состояние
#     element = driver.find_elements(By.CLASS_NAME,'CardInfoRow_state')
#     state = element[0].text.split("\n")[1]

#     # Определяем цену
#     element = driver.find_elements(By.CLASS_NAME,'OfferPriceCaption')
#     price = element[0].text[:-2:].replace(" ","")

#     cars.append({"brand": brand, "model_name": model_name, "bodyType":bodyType, "color":color,\
#         "engineDisplacement":engineDisplacement,'enginePower':enginePower,"fuelType":fuelType,\
#           "mileage":mileage,"productionDate":productionDate,"vehicleTransmission":vehicleTransmission,\
#           "owners":owners,"pts":pts,"wd":wd,"weel":weel,"state":state,"price":price})

#     time.sleep(1)

In [521]:
# DIR_TRAIN  = '../input/all_auto_ru/' # подключаем удаленный датасет к ноутбуку
# DIR_TRAIN_2021  = '../input/parsed_df/' # подключаем спарсенный датасет с auto.ru
# DIR_TEST   = '../input/test_df/'

In [522]:
VAL_SIZE= 0.20  
cols_to_remove = []

In [523]:
!ls '../input'

"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [524]:
train = pd.read_csv('all_auto_ru.csv')  # датасет для обучения модели
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [525]:
pd.options.display.max_columns = None

In [526]:
train.sample(3)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
62911,Внедорожник 5 дв.,SUBARU,0000CC,бензин,2011.0,S-Edition 2.5 AT (263 л.с.) 4WD,5.0,2011,ALLROAD_5_DOORS AUTOMATIC S-Edition,AUTOMATIC,S-Edition,263.0,Авто в эксплуатации с 05.2012 года. Я брал его...,186000,"{'id': '7076515', 'name': 'RU', 'available_opt...",полный,LEFT,NaN,3.0,ORIGINAL,True,"{'year': 2017, 'month': 8}",800000.0,2020-07-01T16:25:04Z,NaN,FORESTER
88865,Внедорожник 5 дв.,SSANG_YONG,EE1D19,дизель,2010.0,2.0d AT (149 л.с.),5.0,2011,ALLROAD_5_DOORS AUTOMATIC 2.0d,AUTOMATIC,2.0d,149.0,"Машина ухоженная, родной окрас, без дтп, в сал...",174400,"{'id': '8519015', 'name': 'Original', 'availab...",передний,LEFT,NaN,2.0,ORIGINAL,True,"{'year': 2017, 'month': 2}",480000.0,2020-07-26T10:49:11Z,NaN,ACTYON
6940,Седан,BMW,040001,бензин,2009.0,528i 3.0 AT (258 л.с.),4.0,2011,SEDAN AUTOMATIC 528i,AUTOMATIC,528i,258.0,Продаю любимый автомобиль. BMW 5-серия в кузов...,134000,{'id': '0'},задний,LEFT,NaN,3.0,ORIGINAL,True,NaN,1250000.0,2020-06-14T20:13:30Z,NaN,5ER


In [527]:
test.sample(3)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
30204,внедорожник 5 дв.,AUDI,https://auto.ru/cars/used/sale/audi/q5/1101307...,красный,NaN,"Приобретался автомобиль в Major Audi Строгино,...",2.0 LTR,225 N12,"{""cruise-control"":true,""tinted-glass"":true,""pt...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,96881,2012,"{""code"":""Q5"",""name"":""Q5"",""ru_name"":""Ку5"",""morp...",Q5,2.0 AT (225 л.с.) 4WD,5,1603569404,RUB,2013,1101307778,"{""id"":""8351307"",""displacement"":1984,""engine_ty...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
4543,седан,AUDI,https://auto.ru/cars/used/sale/audi/a4/1097323...,белый,NaN,Продаю в связи с покупкой нового автомобиля. П...,2.0 LTR,225 N12,"{""18-inch-wheels"":true,""lock"":true}",бензин,https://autoru.naydex.net/ys1kR7800/fbd964zfcB...,72000,2011,"{""code"":""A4"",""name"":""A4"",""ru_name"":""А4"",""morph...",A4,2.0 AMT (225 л.с.) 4WD,4,1603130065,RUB,2015,1097323414,"{""id"":""20033124"",""displacement"":1984,""engine_t...",SEDAN ROBOT 2.0,роботизированная,EUROPEAN,2 владельца,4 года и 6 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен
15748,внедорожник 5 дв.,MERCEDES,https://auto.ru/cars/used/sale/mercedes/gl_kla...,синий,NaN,"Автомобил от официалного дилера, салон не прок...",3.0 LTR,258 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,95000,2012,"{""code"":""GL_KLASSE"",""name"":""GL-Класс"",""ru_name...",GL_KLASSE,350 CDI BlueTEC 3.0d AT (258 л.с.) 4WD,5,1603241325,RUB,2013,1101259222,"{""id"":""8429261"",""name"":""350"",""nameplate"":""350 ...",ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [528]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89378 entries, 0 to 89377
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              89377 non-null  object 
 1   brand                 89378 non-null  object 
 2   color                 89378 non-null  object 
 3   fuelType              89378 non-null  object 
 4   modelDate             89377 non-null  float64
 5   name                  89377 non-null  object 
 6   numberOfDoors         89377 non-null  float64
 7   productionDate        89378 non-null  int64  
 8   vehicleConfiguration  89377 non-null  object 
 9   vehicleTransmission   89377 non-null  object 
 10  engineDisplacement    89377 non-null  object 
 11  enginePower           89377 non-null  float64
 12  description           86124 non-null  object 
 13  mileage               89378 non-null  int64  
 14  Комплектация          89378 non-null  object 
 15  Привод             

In [529]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [530]:
# проанализируем отличие признаков между тестовой и тренировочной выборкой
set(test.columns).difference(train.columns)

{'car_url',
 'complectation_dict',
 'equipment_dict',
 'image',
 'model_info',
 'model_name',
 'parsing_unixtime',
 'priceCurrency',
 'sell_id',
 'super_gen',
 'vendor'}

In [531]:
set(train.columns).difference(test.columns)

{'hidden', 'model', 'price', 'start_date', 'Комплектация'}

Видно, что в данных train есть 4 признака ('hidden', 'model', 'start_date', 'Комплектация') , которых нет в тестовой выборке.
И наоборот в тестовой есть 11 признаков, которых нет в train

### Проанализируем эти признаки для унификации:

#### параметры 'model_info', 'model_name', 'model'

Как видно из таблиц, train.model содержит ту же информацию, что и в test.model_name, поэтому просто переименуем признак.
А признак model_info дублирует model_name, удалим test.model_info.

In [532]:
train.rename(columns={'model': 'model_name'}, inplace=True)
test.drop('model_info', axis=1, inplace=True)

#### параметры 'complectation_dict', 'Комплектация'

Видим, что test.complectation_dict содержит ту же информацию, что и train['Комплектация'], поэтому просто переименуем название признака.

In [533]:
train.rename(columns={'Комплектация': 'complectation_dict'}, inplace=True)

#### параметр 'priceCurrency'

Поскольку признак не несет никакой информации, то его можно удалить.

In [534]:
test.drop('priceCurrency', axis=1, inplace=True)

#### параметр 'parsing_unixtime'

Данный признак содержит даты в диапазоне от 19/10/20 до 26/10/20. Удалим признак, поскольку он не имеет влияния на цену на наш взгляд

In [535]:
test.drop('parsing_unixtime', axis=1, inplace=True)

#### параметры 'sell_id', 'price' 

Добавим нулевые значения в train и test

In [536]:
test['price'] = 0
train['sell_id'] = 0

Удалим оставшиеся признаки, которые есть только в одной из выборок, поскольку они не имееют влияния на цену:

In [537]:
test.drop(['car_url', 'equipment_dict', 'image',
           'super_gen', 'vendor'], axis=1, inplace=True)
train.drop(['hidden', 'start_date'], axis=1, inplace=True)

In [538]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   color                 34686 non-null  object
 3   complectation_dict    6418 non-null   object
 4   description           34686 non-null  object
 5   engineDisplacement    34686 non-null  object
 6   enginePower           34686 non-null  object
 7   fuelType              34686 non-null  object
 8   mileage               34686 non-null  int64 
 9   modelDate             34686 non-null  int64 
 10  model_name            34686 non-null  object
 11  name                  34686 non-null  object
 12  numberOfDoors         34686 non-null  int64 
 13  productionDate        34686 non-null  int64 
 14  sell_id               34686 non-null  int64 
 15  vehicleConfiguration  34686 non-null

Признаки 'Владение'  и 'complectation_dict' имеют большое количество пропусков. Удалим эти столбцы                            

In [539]:
test.drop(['complectation_dict', 'Владение'], axis=1, inplace=True)
train.drop(['complectation_dict', 'Владение'], axis=1, inplace=True)

Рассмотрим, сколько брендов представлено в нашей выборке:

In [540]:
train.brand.sort_values().unique(), test.brand.sort_values().unique()

(array(['AUDI', 'BMW', 'CADILLAC', 'CHERY', 'CHEVROLET', 'CHRYSLER',
        'CITROEN', 'DAEWOO', 'DODGE', 'FORD', 'GEELY', 'GREAT_WALL',
        'HONDA', 'HYUNDAI', 'INFINITI', 'JAGUAR', 'JEEP', 'KIA',
        'LAND_ROVER', 'LEXUS', 'MAZDA', 'MERCEDES', 'MINI', 'MITSUBISHI',
        'NISSAN', 'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'SKODA',
        'SSANG_YONG', 'SUBARU', 'SUZUKI', 'TOYOTA', 'VOLKSWAGEN', 'VOLVO'],
       dtype=object),
 array(['AUDI', 'BMW', 'HONDA', 'INFINITI', 'LEXUS', 'MERCEDES',
        'MITSUBISHI', 'NISSAN', 'SKODA', 'TOYOTA', 'VOLKSWAGEN', 'VOLVO'],
       dtype=object))

Поскольку в тестовой выборке всего 12 брендов, уберем лишние бренды из train

In [541]:
train = train[train.brand.isin(test.brand.unique())]

Признак 'Состояние' содержит только 1 значение, поэтому его можно удалить

In [542]:
test.drop('Состояние', axis=1, inplace=True)
train.drop('Состояние', axis=1, inplace=True)

### Работа с пропусками

In [543]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49309 entries, 0 to 88660
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              49308 non-null  object 
 1   brand                 49309 non-null  object 
 2   color                 49309 non-null  object 
 3   fuelType              49309 non-null  object 
 4   modelDate             49308 non-null  float64
 5   name                  49308 non-null  object 
 6   numberOfDoors         49308 non-null  float64
 7   productionDate        49309 non-null  int64  
 8   vehicleConfiguration  49308 non-null  object 
 9   vehicleTransmission   49308 non-null  object 
 10  engineDisplacement    49308 non-null  object 
 11  enginePower           49308 non-null  float64
 12  description           47721 non-null  object 
 13  mileage               49309 non-null  int64  
 14  Привод                49308 non-null  object 
 15  Руль               

In [544]:
# пропуски в графе 'Владельцы' можно заменить на наиболее часто повторяющееся значение
train['Владельцы'].value_counts()

3.0    19107
2.0    10270
1.0     8962
Name: Владельцы, dtype: int64

In [545]:
train['Владельцы'].fillna(3, inplace=True)

In [546]:
# удалим признак description
test.drop('description', axis=1, inplace=True)
train.drop('description', axis=1, inplace=True)

In [547]:
train.dropna(subset=['price'], inplace=True)

In [548]:
# заменим пропуски в 'ПТС' на 'Оригинал'
train['ПТС'].fillna('Оригинал', inplace=True)

In [549]:
train.dropna(inplace=True)

In [550]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49100 entries, 0 to 88660
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              49100 non-null  object 
 1   brand                 49100 non-null  object 
 2   color                 49100 non-null  object 
 3   fuelType              49100 non-null  object 
 4   modelDate             49100 non-null  float64
 5   name                  49100 non-null  object 
 6   numberOfDoors         49100 non-null  float64
 7   productionDate        49100 non-null  int64  
 8   vehicleConfiguration  49100 non-null  object 
 9   vehicleTransmission   49100 non-null  object 
 10  engineDisplacement    49100 non-null  object 
 11  enginePower           49100 non-null  float64
 12  mileage               49100 non-null  int64  
 13  Привод                49100 non-null  object 
 14  Руль                  49100 non-null  object 
 15  Владельцы          

In [551]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   color                 34686 non-null  object
 3   engineDisplacement    34686 non-null  object
 4   enginePower           34686 non-null  object
 5   fuelType              34686 non-null  object
 6   mileage               34686 non-null  int64 
 7   modelDate             34686 non-null  int64 
 8   model_name            34686 non-null  object
 9   name                  34686 non-null  object
 10  numberOfDoors         34686 non-null  int64 
 11  productionDate        34686 non-null  int64 
 12  sell_id               34686 non-null  int64 
 13  vehicleConfiguration  34686 non-null  object
 14  vehicleTransmission   34686 non-null  object
 15  Владельцы             34686 non-null

Добавим в выборку файл с собранными данными

In [552]:
train_2021 = pd.read_csv('parsed_df.csv')
train_2021.sample(3)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,views,date_added,region,price
20495,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_clas...,чёрный,"['cruise-control', 'multi-wheel', 'heated-wash...",Выгода до 100 000 рублей при покупке в кредит...,4.7,455.0,"{'e-adjustment-wheel': True, 'multi-wheel': Tr...",Бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,102000.0,Maybach S-Класс,Mercedes-Benz Maybach S-Класс 500 I (X222),4.0,1.640236e+09,RUB,2016.0,1.106337e+09,"{'sale-data-attributes': {'asciiCat': 'cars', ...",AUTOMATIC,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,182.0,17 декабря,в Москве,6050000.0
10371,внедорожник,MERCEDES,https://auto.ru/cars/used/sale/mercedes/m_klas...,чёрный,NaN,Ваш выбор и Ваше время наши главные приоритеты...,3.2,218.0,"{'cruise-control': True, 'asr': True, 'airbag-...",Бензин,https://autoru.naydex.net/a1cC78K01/f6f2b5RS/Q...,290000.0,M-Класс,Mercedes-Benz M-Класс 320 I (W163) Рестайлинг,5.0,1.640222e+09,RUB,2002.0,1.105823e+09,"{'sale-data-attributes': {'asciiCat': 'cars', ...",AUTOMATIC,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,390.0,22 декабря,в Москве,560000.0
26952,купе-хардтоп,MERCEDES,https://auto.ru/cars/used/sale/mercedes/e_klas...,белый,"['cruise-control', 'multi-wheel', 'auto-park',...",Продаю свой автомобиль технически всё в идеале...,2.0,184.0,"{'cruise-control': True, 'esp': True, 'adaptiv...",Бензин,https://autoru.naydex.net/1KpQ7K631/e24d99_tl/...,117000.0,E-Класс,"Mercedes-Benz E-Класс 200 IV (W212, S212, C207...",2.0,1.640245e+09,RUB,2014.0,1.106369e+09,"{'sale-data-attributes': {'asciiCat': 'cars', ...",AUTOMATIC,3 или более,1 год и 7 месяцев,Дубликат,задний,Левый,Не требует ремонта,Растаможен,233.0,21 декабря,в Москве,1960000.0


Найдем отличия с тестовой таблицей

In [553]:
set(test.columns).difference(train_2021.columns)

{'modelDate', 'vehicleConfiguration'}

In [554]:
pars = train_2021.copy()
pars.drop(['car_url', 'equipment_dict', 'image', 'super_gen', 'priceCurrency', 'parsing_unixtime', 'complectation_dict',
           'Владение', 'Состояние', 'description', 'views', 'date_added',  'region'], axis=1, inplace=True)

In [555]:
pars.dropna(subset=['price'], inplace=True)

In [556]:
pars.dropna(subset=['fuelType'], inplace=True)

In [557]:
pars.dropna(inplace=True)

In [558]:
pars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28719 entries, 0 to 28899
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   bodyType             28719 non-null  object 
 1   brand                28719 non-null  object 
 2   color                28719 non-null  object 
 3   engineDisplacement   28719 non-null  object 
 4   enginePower          28719 non-null  float64
 5   fuelType             28719 non-null  object 
 6   mileage              28719 non-null  float64
 7   model_name           28719 non-null  object 
 8   name                 28719 non-null  object 
 9   numberOfDoors        28719 non-null  float64
 10  productionDate       28719 non-null  float64
 11  sell_id              28719 non-null  float64
 12  vehicleTransmission  28719 non-null  object 
 13  Владельцы            28719 non-null  object 
 14  ПТС                  28719 non-null  object 
 15  Привод               28719 non-null 

In [559]:
# удалим повторы
set(pars.columns).difference(test.columns)

set()

# Предварительная обработка данных и EDA

### Объединим train и test датасеты

In [560]:
# добавим столбец, который будет показывать train или test, чтобы было легче их объединять и разделять
train['train'] = 1
pars['train'] = 1
test['train'] = 0

# эту колонку следует добавить к train, потому что она есть в test, и используется для submission
train['sell_id'] = 0

#train_2021['sell_id'] = 0
pars['sell_id'] = 0

In [561]:
combined_df = pd.concat([test, train, pars], join='inner', ignore_index=True)
print(combined_df.shape)

(112505, 20)


In [562]:
test.columns

Index(['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower',
       'fuelType', 'mileage', 'modelDate', 'model_name', 'name',
       'numberOfDoors', 'productionDate', 'sell_id', 'vehicleConfiguration',
       'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'Таможня',
       'price', 'train'],
      dtype='object')

### Построим наивную модель

Сначала изменим категориальные признаки

In [563]:
for col in ['brand', 'model_name']:
    combined_df[col] = combined_df[col].astype('category').cat.codes

In [564]:
naiv = combined_df.columns[combined_df.dtypes != object]

In [565]:
naiv = combined_df[naiv]

In [566]:
naiv['age'] = 2021-naiv['productionDate']

In [567]:
naiv[naiv.sell_id == 0]

,brand,mileage,model_name,numberOfDoors,productionDate,sell_id,price,train,age
34686,0,350000.0,2,4.0,1991.0,0,200000.0,1,30.0
34687,0,173424.0,2,4.0,1986.0,0,60000.0,1,35.0
34688,0,230000.0,2,5.0,1989.0,0,99000.0,1,32.0
34689,0,240000.0,2,4.0,1989.0,0,65000.0,1,32.0
34690,0,300000.0,2,4.0,1991.0,0,100000.0,1,30.0
...,...,...,...,...,...,...,...,...,...
112500,7,207000.0,757,4.0,2015.0,0,790000.0,1,6.0
112501,9,201000.0,459,5.0,2013.0,0,3400000.0,1,8.0
112502,8,218000.0,546,5.0,2013.0,0,1220000.0,1,8.0
112503,0,99000.0,798,2.0,2016.0,0,2450000.0,1,5.0


In [568]:
X = naiv[naiv.sell_id == 0].drop(['price', 'train', 'productionDate'], axis=1)
y = naiv[naiv.sell_id == 0].price

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=VAL_SIZE,  random_state=RANDOM_SEED)

lr = LinearRegression().fit(X_train, y_train)
y_pred = (lr.predict(X_test))

print(
    f"The accuracy of the naive model using MAPE metrics is : {(mape(y_test, y_pred))*100:0.2f}%.")

The accuracy of the naive model using MAPE metrics is : 130.11%.


Наивная модель дала результат 130. С ним будем сравнивать результаты других моделей. 

#### Рассмотрим каждый признак и, по возможности, заменим категориальный признак на числовой или бинарный

In [569]:
combined_df.sample(3)

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,model_name,name,numberOfDoors,productionDate,sell_id,vehicleTransmission,Владельцы,ПТС,Привод,Руль,Таможня,price,train
89962,лифтбек,8,серый,1.6,105,Дизель,157000.0,546,Skoda Octavia II (A5) Рестайлинг,5.0,2011.0,0,MECHANICAL,3 или более,Оригинал,передний,Левый,Растаможен,570000.0,1
89399,внедорожник,7,красный,1.6,117,Бензин,89000.0,424,Nissan Juke I,5.0,2011.0,0,VARIATOR,2 владельца,Оригинал,передний,Левый,Растаможен,1140000.0,1
87528,хэтчбек,5,голубой,2.0,211,Бензин,101000.0,54,Mercedes-Benz A-Класс 250 III (W176),5.0,2013.0,0,ROBOT,1 владелец,Оригинал,полный,Левый,Растаможен,1370000.0,1


In [570]:
combined_df.bodyType.unique()

array(['лифтбек', 'внедорожник 5 дв.', 'хэтчбек 5 дв.', 'седан',
       'компактвэн', 'универсал 5 дв.', 'пикап одинарная кабина',
       'хэтчбек 3 дв.', 'купе', 'кабриолет', 'минивэн',
       'пикап двойная кабина', 'внедорожник 3 дв.', 'родстер', 'микровэн',
       'седан 2 дв.', 'купе-хардтоп', 'фастбек', 'тарга',
       'внедорожник открытый', 'лимузин', 'пикап полуторная кабина',
       'седан-хардтоп', 'фургон', 'Седан', 'Универсал 5 дв.',
       'Хэтчбек 5 дв. Sportback', 'Хэтчбек 3 дв.', 'Хэтчбек 5 дв.',
       'Кабриолет', 'Купе', 'Лифтбек Sportback', 'Лифтбек', 'Седан Long',
       'Внедорожник 5 дв.', 'Кабриолет Roadster', 'Седан 2 дв.',
       'Седан Gran Coupe', 'Компактвэн', 'Компактвэн Gran Tourer',
       'Лифтбек Gran Turismo', 'Хэтчбек 3 дв. Compact',
       'Лифтбек Gran Coupe', 'Купе-хардтоп', 'Родстер Roadster',
       'Родстер', 'Седан Type-S', 'Микровэн', 'Универсал 5 дв. Shuttle',
       'Родстер del Sol', 'Минивэн', 'Компактвэн Spike',
       'Внедорожник 3 дв

In [571]:
combined_df.bodyType = combined_df.bodyType.apply(
    lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)

In [572]:
combined_df.bodyType.unique()

array(['лифтбек', 'внедорожник', 'хэтчбек', 'седан', 'компактвэн',
       'универсал', 'пикап', 'купе', 'кабриолет', 'минивэн', 'родстер',
       'микровэн', 'купе-хардтоп', 'фастбек', 'тарга', 'лимузин',
       'седан-хардтоп', 'фургон'], dtype=object)

In [573]:
combined_df.bodyType = combined_df['bodyType'].astype('category').cat.codes

#### признак "цвет"

In [574]:
combined_df.color.value_counts()

чёрный         20325
040001         16135
белый          12427
FAFBFB          8988
серый           7693
серебристый     6444
синий           5957
97948F          5622
CACECB          5496
0000CC          4963
коричневый      2910
красный         2655
200204          2028
EE1D19          1784
зелёный         1455
007F00          1321
бежевый         1080
C49648           857
голубой          675
22A0F8           591
пурпурный        445
золотистый       387
фиолетовый       360
жёлтый           334
DEA522           317
660099           314
4A2197           248
оранжевый        241
FFD600           224
FF8649           199
розовый           17
FFC0CB            13
Name: color, dtype: int64

In [575]:
# используем словарь для перевода кодировки цветов:
color_dict = {'040001': 'чёрный', 'FAFBFB': 'белый', '97948F': 'серый', 'CACECB': 'серебристый', '0000CC': 'синий', '200204': 'коричневый',
              'EE1D19': 'красный',  '007F00': 'зелёный', 'C49648': 'бежевый', '22A0F8': 'голубой', '660099': 'пурпурный', 'DEA522': 'золотистый',
              '4A2197': 'фиолетовый', 'FFD600': 'жёлтый', 'FF8649': 'оранжевый', 'FFC0CB': 'розовый'}
combined_df.color.replace(color_dict, inplace=True)
combined_df.color.value_counts(normalize=True)

чёрный         0.324074
белый          0.190347
серый          0.118350
серебристый    0.106129
синий          0.097062
коричневый     0.043891
красный        0.039456
зелёный        0.024674
бежевый        0.017217
голубой        0.011253
пурпурный      0.006746
золотистый     0.006257
фиолетовый     0.005404
жёлтый         0.004960
оранжевый      0.003911
розовый        0.000267
Name: color, dtype: float64

In [576]:
for col in ['color']:
    combined_df[col] = combined_df[col].astype('category').cat.codes

#### признак "рабочий объем"

In [577]:
combined_df.engineDisplacement = combined_df.name.apply(
    transf_engineDisplacement_to_float)

In [578]:
combined_df.engineDisplacement.unique()

array([1.2, 1.6, 1.8, 2. , 1.4, 1.3, 1. , 3.6, 1.5, 1.9, 2.8, 1.1, 2.5,
       4.2, 3. , 4. , 5.9, 2.7, 3.1, 2.4, 5.2, 3.2, 4.1, 6.3, 2.3, 6. ,
       2.2, 3.7, 2.9, 5. , 3.3, 2.1, 2.6, nan, 3.5, 1.7, 0.7, 4.4, 4.8,
       5.4, 6.6, 4.9, 3.8, 3.4, 3.9, 4.6, 5.6, 4.5, 5.5, 6.2, 4.7, 4.3,
       5.8, 5.3, 5.7, 0. ])

In [579]:
test[test.brand == 'MERCEDES'].engineDisplacement.unique()

array(['2.1 LTR', '1.6 LTR', '2.0 LTR', '1.8 LTR', '3.0 LTR', '5.5 LTR',
       '3.5 LTR', '6.2 LTR', '4.0 LTR', '2.9 LTR', '4.7 LTR', '4.2 LTR',
       '2.5 LTR', '2.3 LTR', '1.7 LTR', '2.2 LTR', '6.0 LTR', '4.3 LTR',
       '5.0 LTR', '1.3 LTR', '3.2 LTR', '2.6 LTR', '1.5 LTR', '2.4 LTR',
       '5.4 LTR', '2.8 LTR', '2.7 LTR', '3.7 LTR', '4.5 LTR', '5.6 LTR',
       '5.8 LTR', '5.3 LTR', '1.9 LTR', '3.8 LTR', '1.4 LTR', ' LTR'],
      dtype=object)

In [580]:
combined_df[combined_df.engineDisplacement.isna()].fuelType.unique()

array(['электро', 'Бензин', 'Дизель',
       'Бензин, газобаллонное оборудование', 'Гибрид',
       'Дизель, газобаллонное оборудование', 'Газ',
       'Гибрид, газобаллонное оборудование',
       'Газ, газобаллонное оборудование'], dtype=object)

In [581]:
# так как значения nan у признака "рабочий объем" есть только у электромашин, то заполним значения 0
combined_df.engineDisplacement.fillna(0, inplace=True)

#### признак "тип топлива"

In [582]:
combined_df.fuelType.value_counts()

бензин                                67863
Бензин                                18899
дизель                                15264
Дизель                                 8801
Гибрид                                  724
гибрид                                  491
Бензин, газобаллонное оборудование      170
электро                                 151
Газ                                      79
Дизель, газобаллонное оборудование       43
газ                                      17
Газ, газобаллонное оборудование           2
Гибрид, газобаллонное оборудование        1
Name: fuelType, dtype: int64

In [583]:
# разделим этот признак на две группы: бензин и не бензин
combined_df.fuelType = combined_df.fuelType.apply(
    lambda x: 1 if x == 'бензин' else 0)

In [584]:
combined_df.fuelType.value_counts()

1    67863
0    44642
Name: fuelType, dtype: int64

#### признак "дата производства"

In [585]:
# заменим дату на возраст
combined_df['age'] = 2021 - combined_df.productionDate

In [586]:
combined_df['age']

0         7.0
1         4.0
2         7.0
3         7.0
4         9.0
         ... 
112500    6.0
112501    8.0
112502    8.0
112503    5.0
112504    4.0
Name: age, Length: 112505, dtype: float64

#### признак "тип коробки передач"

In [587]:
combined_df.vehicleTransmission.value_counts()

AUTOMATIC           43670
автоматическая      19596
MECHANICAL          16161
ROBOT                9136
VARIATOR             8852
механическая         7209
вариатор             3999
роботизированная     3882
Name: vehicleTransmission, dtype: int64

In [588]:
automat = ['AUTOMATIC', 'автоматическая',  'VARIATOR',
           'ROBOT', 'вариатор', 'роботизированная']
mechanic = ['MECHANICAL', 'механическая']

In [589]:
# разделим параметры на две группы: механическая и автоматическая
combined_df.vehicleTransmission = combined_df.vehicleTransmission.apply(
    lambda x: 1 if x in automat else 0)

In [590]:
combined_df.vehicleTransmission.value_counts()

1    89135
0    23370
Name: vehicleTransmission, dtype: int64

#### параметр "владельцы"

In [591]:
print(combined_df['Владельцы'].unique())

['3 или более' '1\xa0владелец' '2\xa0владельца' 3.0 1.0 2.0 '1 владелец'
 '2 владельца']


In [592]:
combined_df['owners'] = combined_df['Владельцы'].apply(transf_owners_to_float)
combined_df.owners.unique()

array([3., 1., 2.])

#### признак "птс"

In [593]:
print(combined_df['ПТС'].unique())

['Оригинал' 'Дубликат' nan 'ORIGINAL' 'DUPLICATE']


In [594]:
combined_df['ПТС'].value_counts()

Оригинал     55382
ORIGINAL     43214
Дубликат      8284
DUPLICATE     5624
Name: ПТС, dtype: int64

In [595]:
combined_df.drop(['ПТС', 'Владельцы'], axis=1, inplace=True)

#### признак "трансмиссия"

In [596]:
print(combined_df['Привод'].unique())

['передний' 'полный' 'задний']


In [597]:
combined_df['Привод'].value_counts()

полный      52219
передний    45810
задний      14476
Name: Привод, dtype: int64

In [598]:
# присвоим численные значения категориальному признаку
combined_df['transmission '] = combined_df['Привод'].apply(
    lambda x: 1 if x == 'полный' else 2 if x == 'передний' else 3)

In [599]:
combined_df.drop(['Привод', 'Руль', 'Таможня'], axis=1, inplace=True)

In [600]:
data_col = combined_df.columns[combined_df.dtypes != object]
data = combined_df[data_col]
data

,bodyType,brand,color,engineDisplacement,fuelType,mileage,model_name,numberOfDoors,productionDate,sell_id,vehicleTransmission,price,train,age,owners,transmission
0,6,8,13,1.2,1,74000.0,537,5.0,2014.0,1100575026,1,0.0,0,7.0,3.0,2
1,6,8,15,1.6,1,60563.0,537,5.0,2017.0,1100549428,0,0.0,0,4.0,1.0,2
2,6,8,12,1.8,1,88000.0,748,5.0,2014.0,1100658222,1,0.0,0,7.0,1.0,2
3,6,8,6,1.6,1,95000.0,537,5.0,2014.0,1100937408,1,0.0,0,7.0,1.0,2
4,6,8,1,1.8,1,58536.0,537,5.0,2012.0,1101037972,1,0.0,0,9.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112500,11,7,11,0.0,0,207000.0,757,4.0,2015.0,0,1,790000.0,1,6.0,1.0,2
112501,0,9,15,0.0,0,201000.0,459,5.0,2013.0,0,1,3400000.0,1,8.0,3.0,1
112502,6,8,1,0.0,0,218000.0,546,5.0,2013.0,0,0,1220000.0,1,8.0,3.0,1
112503,3,0,13,0.0,0,99000.0,798,2.0,2016.0,0,0,2450000.0,1,5.0,1.0,2


### Linear Regression

In [601]:
X = data[data.sell_id == 0].drop(
    ['price', 'train', 'productionDate'], axis=1)
y = data[data.sell_id == 0].price

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=VAL_SIZE,  random_state=RANDOM_SEED)

lr = LinearRegression().fit(X_train, y_train)
y_pred = (lr.predict(X_test))

print(
    f"The accuracy of the naive model using MAPE metrics is : {(mape(y_test, y_pred))*100:0.2f}%.")

The accuracy of the naive model using MAPE metrics is : 130.03%.


Линейная регрессия не показала никаких улучшений. Поработаем с выбросами и проверим результат.

#### Работа с выбросами

In [602]:
combined_df.sample(3)

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,model_name,name,numberOfDoors,productionDate,sell_id,vehicleTransmission,price,train,age,owners,transmission
49674,11,6,4,1.6,103,1,200000.0,153,1.6 MT (103 л.с.),4.0,2001.0,0,0,175000.0,1,20.0,3.0,2
96188,11,6,11,0.0,90,0,196000.0,457,Mitsubishi Lancer IX Рестайлинг,4.0,2006.0,0,0,600000.0,1,15.0,3.0,2
15305,11,3,15,3.5,315 N12,1,190000.0,329,G35 3.5 AT (315 л.с.) 4WD,4.0,2008.0,1099152052,1,0.0,0,13.0,3.0,1


In [603]:
for col in data.columns:
    display(vis_num_feature(combined_df, col, 'price', 'train == 1'))
    display(calculate_stat_outliers(combined_df, col, log=True))
    print('\n' + '-' * 10 + '\n')

None

,left,right
borders,-5.378906,8.964844
# outliers,0.000000,0.000000



----------



None

,left,right
borders,-1.020996,5.928223
# outliers,0.000000,0.000000


...

,left,right
borders,0.962406,2.622556
# outliers,0.000000,0.000000



----------



None

,left,right
borders,0.122556,2.462406
# outliers,0.000000,0.000000



----------



Признак 'enginePower' имеет выбросы, избавимся от них.

In [604]:
combined_df.enginePower = combined_df.enginePower.apply(
    transf_enginePower_to_float)

In [606]:
combined_df.enginePower

0         105.0
1         110.0
2         152.0
3         110.0
4         152.0
          ...  
112500    117.0
112501    235.0
112502    150.0
112503    184.0
112504    249.0
Name: enginePower, Length: 112505, dtype: float64

In [607]:
combined_df[combined_df.enginePower > 640]

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,model_name,name,numberOfDoors,productionDate,sell_id,vehicleTransmission,price,train,age,owners,transmission
79051,0,5,15,4.0,700.0,1,0.0,379,Brabus 700 4.0 AT (700 л.с.) 4WD,5.0,2020.0,0,1,29300000.0,1,1.0,3.0,1
79118,0,5,15,4.0,800.0,1,0.0,379,Brabus 800 4.0 AT (800 л.с.) 4WD,5.0,2020.0,0,1,31600000.0,1,1.0,3.0,1
92916,0,5,13,0.0,700.0,0,89000.0,328,Mercedes-Benz G-Класс AMG Brabus 700 I (W463) ...,5.0,2013.0,0,1,6440000.0,1,8.0,3.0,1
97660,0,5,15,0.0,700.0,0,52000.0,328,Mercedes-Benz G-Класс AMG Brabus 700 I (W463) ...,5.0,2015.0,0,1,16200000.0,1,6.0,2.0,1
98503,0,5,15,0.0,800.0,0,3000.0,328,Mercedes-Benz G-Класс AMG Brabus 800 II (W463),5.0,2020.0,0,1,21530000.0,1,1.0,1.0,1
99068,0,5,7,0.0,800.0,0,29000.0,328,Mercedes-Benz G-Класс AMG Brabus 800 II (W463),5.0,2019.0,0,1,16640000.0,1,2.0,1.0,1
99350,0,5,12,0.0,800.0,0,7000.0,328,Mercedes-Benz G-Класс AMG Brabus 800 II (W463),5.0,2021.0,0,1,24280000.0,1,0.0,1.0,1
99667,0,5,15,0.0,800.0,0,11000.0,328,Mercedes-Benz G-Класс AMG Brabus 800 II (W463),5.0,2020.0,0,1,21650000.0,1,1.0,1.0,1
99774,0,5,12,0.0,700.0,0,95000.0,328,Mercedes-Benz G-Класс AMG Brabus 700 I (W463) ...,5.0,2015.0,0,1,6370000.0,1,6.0,2.0,1
100157,0,5,15,0.0,800.0,0,1000.0,328,Mercedes-Benz G-Класс AMG Brabus 800 II (W463),5.0,2021.0,0,1,22680000.0,1,0.0,1.0,1


In [608]:
# удалим строки из train
combined_df = combined_df[combined_df.enginePower < 640]

#### параметр "возраст машины"

In [609]:
combined_df[combined_df['train'] == 1].age.sort_values()

97749       0.0
107490      0.0
104070      0.0
107492      0.0
98928       0.0
          ...  
91784      84.0
40533      84.0
40528      84.0
83777      85.0
83194     117.0
Name: age, Length: 77790, dtype: float64

In [610]:
sd = combined_df[combined_df['train'] == 0]
sd[sd.age > 80]

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,model_name,name,numberOfDoors,productionDate,sell_id,vehicleTransmission,price,train,age,owners,transmission
1777,17,8,15,1.1,30.0,1,14000.0,572,1.1 MT (30 л.с.),3.0,1939.0,1097046168,0,0.0,0,82.0,3.0,3
3174,11,0,15,3.3,75.0,1,90000.0,51,3.3 MT (75 л.с.),4.0,1938.0,1091525266,0,0.0,0,83.0,2.0,3
9625,11,1,0,2.0,51.0,1,16000.0,26,2.0 MT (51 л.с.),4.0,1937.0,1094373106,0,0.0,0,84.0,2.0,3
9819,11,1,15,2.0,46.0,1,4500.0,25,2.0 MT (46 л.с.),2.0,1937.0,1040149625,0,0.0,0,84.0,3.0,3
16891,11,5,7,1.7,38.0,1,1.0,870,1.7 MT (38 л.с.),4.0,1936.0,1099428966,0,0.0,0,85.0,1.0,3
16944,1,5,1,5.3,32.0,1,48000.0,711,5.3 MT (32 л.с.),0.0,1904.0,1093802104,0,0.0,0,117.0,1.0,3


In [611]:
plt.figure(figsize=(25, 6))
sns.scatterplot(
    data=combined_df[combined_df['train'] == 1], x='age', y="price")

<AxesSubplot:xlabel='age', ylabel='price'>

#### параметр "цена"

In [612]:
combined_df.query('train == 1').price.hist()
plt.title('The target variable distribution', fontdict={'fontsize': 14})
plt.xlabel('price ')

Text(0.5, 0, 'price ')

Распределение имеет тяжелый левый хвост,чтобы это исправить воспользуемся логорифмированием.

In [613]:
np.log2(combined_df.query('train == 1').price).hist()
plt.title('The log2 target variable distribution', fontdict={'fontsize': 14})

Text(0.5, 1.0, 'The log2 target variable distribution')

In [614]:
# добавим новый признак
combined_df['price_log2'] = np.log2(combined_df.price + 1)

# Feature engineering

Создадим новые признаки:

- mileage_per_year: с помощью the productionDate и mileage columns получим информацию,сколько км проехал автомобиль за год;
- rarity: был ли автомобиль произведен ранее 1960;
- older_3y: старше ли автомобиль трёх лет;
- older_5y: старше ли автомобиль пяти лет;

In [615]:
combined_df['mileage_per_year'] = combined_df.mileage / combined_df.age
combined_df['rarity'] = combined_df.productionDate.apply(
    lambda x: 1 if x < 1960 else 0)
combined_df['older_3y'] = combined_df.age.apply(lambda x: 1 if x > 3 else 0)
combined_df['older_5y'] = combined_df.age.apply(lambda x: 1 if x > 5 else 0)

In [616]:
combined_df.mileage_per_year = combined_df.apply(
    lambda x: x.mileage if x.age == 0 else x.mileage / x.age, axis=1)
combined_df[combined_df.age == 0]

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,model_name,name,numberOfDoors,productionDate,sell_id,vehicleTransmission,price,train,age,owners,transmission,price_log2,mileage_per_year,rarity,older_3y,older_5y
97395,0,5,15,0.0,224.0,0,12000.0,334,Mercedes-Benz GLA 250 II (H247),5.0,2021.0,0,1,3900000.0,1,0.0,1.0,2,21.895043,12000.0,0,0,0
97398,11,5,15,0.0,249.0,0,7000.0,680,Mercedes-Benz S-Класс 350 d Long 4MATIC VII (W...,4.0,2021.0,0,1,15340000.0,1,0.0,1.0,1,23.870795,7000.0,0,0,0
97400,6,8,1,0.0,110.0,0,5000.0,675,Skoda Rapid II,5.0,2021.0,0,1,1330000.0,1,0.0,2.0,2,20.342996,5000.0,0,0,0
97415,0,1,1,0.0,292.0,0,13000.0,888,BMW X3 30e xDrive III (G01),5.0,2021.0,0,1,5690000.0,1,0.0,2.0,1,22.439997,13000.0,0,0,0
97420,0,1,15,0.0,184.0,0,8000.0,890,BMW X4 20i II (G02),5.0,2021.0,0,1,5710000.0,1,0.0,1.0,1,22.445060,8000.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112474,0,9,15,0.0,222.0,0,1000.0,855,Toyota Venza II,5.0,2021.0,0,1,5620000.0,1,0.0,1.0,1,22.422139,1000.0,0,0,0
112478,8,5,15,0.0,190.0,0,28000.0,823,Mercedes-Benz V-Класс 250 d длинный II,5.0,2021.0,0,1,9610000.0,1,0.0,1.0,3,23.196105,28000.0,0,0,0
112479,0,9,6,0.0,149.0,0,3000.0,648,Toyota RAV4 V (XA50),5.0,2021.0,0,1,3730000.0,1,0.0,1.0,1,21.830745,3000.0,0,0,0
112490,0,9,15,0.0,166.0,0,6000.0,322,Toyota Fortuner II Рестайлинг,5.0,2021.0,0,1,4590000.0,1,0.0,1.0,1,22.130063,6000.0,0,0,0


In [617]:
combined_df.query('train == 1').mileage_per_year.hist(bins=10)
plt.title('mileage_per_year distribution', fontdict={'fontsize': 14})

Text(0.5, 1.0, 'mileage_per_year distribution')

Снова тяжелый хвост. Логорифмируем:

In [618]:
np.log2(combined_df.query('train == 1').mileage_per_year+1).hist()
plt.title('The log2 mileage_per_year distribution', fontdict={'fontsize': 14})

Text(0.5, 1.0, 'The log2 mileage_per_year distribution')

In [619]:
# добавим новый признак
combined_df['mileage_per_year_log2'] = np.log2(
    combined_df.mileage_per_year + 1)

In [620]:
combined_df.enginePower.hist(bins=10)

<AxesSubplot:title={'center':'The log2 mileage_per_year distribution'}, xlabel='price ', ylabel='price'>

In [621]:
combined_df.columns

Index(['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower',
       'fuelType', 'mileage', 'model_name', 'name', 'numberOfDoors',
       'productionDate', 'sell_id', 'vehicleTransmission', 'price', 'train',
       'age', 'owners', 'transmission ', 'price_log2', 'mileage_per_year',
       'rarity', 'older_3y', 'older_5y', 'mileage_per_year_log2'],
      dtype='object')

In [622]:
temp1 = combined_df.copy()
temp1.sample(2)

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,model_name,name,numberOfDoors,productionDate,sell_id,vehicleTransmission,price,train,age,owners,transmission,price_log2,mileage_per_year,rarity,older_3y,older_5y,mileage_per_year_log2
48861,0,4,15,4.7,234.0,1,376300.0,456,470 4.7 AT (234 л.с.) 4WD,5.0,2003.0,0,1,1200000.0,1,18.0,3.0,1,20.194604,20905.555556,0,1,1,14.351668
31840,0,5,15,3.0,224.0,0,136000.0,362,350 3.0d AT (224 л.с.) 4WD,5.0,2010.0,1092204994,1,0.0,0,11.0,2.0,1,0.000000,12363.636364,0,1,1,13.593932


# Построение моделей ML

In [623]:
X = combined_df.query('train == 1').drop(
    ['price', 'train', 'name', 'price_log2', 'mileage_per_year'], axis=1)

y = combined_df.query('train == 1').price
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [624]:
lr = LinearRegression().fit(X_train, y_train)
y_pred = (lr.predict(X_test))

print(
    f"The accuracy of the naive model using MAPE metrics is : {(mape(y_test, y_pred))*100:0.2f}%.")

The accuracy of the naive model using MAPE metrics is : 91.09%.


Работа с данными привела к улучшению результата модели (MAPE 91.09%), однако на практике ее применение не несёт особой пользы. Построим другие модели

### RandomForestRegressor

In [625]:
rf = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf.fit(X_train, y_train)
predict_rf = rf.predict(X_test)

print(
    f"The MAPE mertics of the Random Forest model using MAPE metrics: {(mape(y_test, predict_rf) * 100):0.2f}%.")

# with log-transformation of the target variable
rf_log = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_log.fit(X_train, np.log(y_train))
predict_rf_log = np.exp(rf_log.predict(X_test))

print(
    f"The MAPE mertic for the Random Forest model is : {(mape(y_test, predict_rf_log) * 100):0.2f}%.")

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


The MAPE mertics of the Random Forest model using MAPE metrics: 15.60%.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.2s


The MAPE mertic for the Random Forest model is : 14.20%.


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.8s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


Результат MAPE 14.20%

### ExtraTreesRegressor

In [626]:
X = combined_df.query('train == 1').drop(
    ['price', 'train', 'name', 'price_log2', 'mileage_per_year'], axis=1)
y = combined_df.query('train == 1').price
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [627]:
X1 = np.array(X)
y1 = np.array(y)

etr_log = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)

skf = KFold(n_splits=4)
etr_log_mape_values = []

for train_index, test_index in skf.split(X1, y1):
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y1[train_index], y1[test_index]

    etr_log.fit(X_train, np.log(y_train))
    y_pred = np.exp(etr_log.predict(X_test))

    etr_log_mape_value = mape(y_test, y_pred)
    etr_log_mape_values.append(etr_log_mape_value)
    print(etr_log_mape_value)

print(
    f"The MAPE mertic for the default ExtraTreesRegressor model using 4-fold CV is: {(np.mean(etr_log_mape_values) * 100):0.2f}%.")

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.6s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


0.23522151932937024


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.6s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


0.22432219222206354


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.5s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


0.21979568507388886


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.8s


0.22746392840899593
The MAPE mertic for the default ExtraTreesRegressor model using 4-fold CV is: 22.67%.


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.5s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


В данном случае результат MAPE 22.67%

### XGBoostRegressor

In [628]:
X1 = np.array(X)
y1 = np.array(y)

xgb_log = xgb.XGBRegressor(max_depth=10,
                           n_estimators=1000,
                           random_state=RANDOM_SEED,
                           n_jobs=-1)
skf = KFold(n_splits=4)
xgb_log_mape_values = []

for train_index, test_index in skf.split(X1, y1):
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y1[train_index], y1[test_index]

    xgb_log.fit(X_train, np.log(y_train))
    y_pred = np.exp(xgb_log.predict(X_test))

    xgb_log_mape_value = mape(y_test, y_pred)
    xgb_log_mape_values.append(xgb_log_mape_value)
    print(xgb_log_mape_value)

print(
    f"The MAPE mertic for the XGBRegressor model using 4-fold CV: {(np.mean(xgb_log_mape_values) * 100):0.2f}%.")

0.2617275655558588
0.23335829934430205
0.2065703396125253
0.188326516987709
The MAPE mertic for the XGBRegressor model using 4-fold CV: 22.25%.


Здесь метрика MAPE 22.25%

### StackingRegressor

In [629]:
estimators = [
    ('etr', ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)),
    ('xgb', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.05,
                             max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000, random_state=RANDOM_SEED))]

sr_log = StackingRegressor(estimators=estimators,
                           final_estimator=LinearRegression())

sr_log.fit(X_train, np.log(y_train))

y_pred = np.exp(sr_log.predict(X_test))

print(
    f"The MAPE mertic for the default StackingRegressor model: {(mape(y_test, y_pred) * 100):0.2f}%.")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.8s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n

The MAPE mertic for the default StackingRegressor model: 19.17%.


MAPE 19.17%

# Выводы по моделям:

Лучший результат метрики MAPE показала модель RandomForestRegressor(14.20%), а худший - ExtraTreesRegressor с результатом 22.67%. Средние результаты у моделей XGBoostRegressor и StackingRegressor (22.25% и 19.17% соответственно). Предположительно, высокий показатель метрики у RandomForestRegressor стал результатом переобучения модели. Поэтому в качестве модели для final submission был выбран стэкинг.

# Submission

In [636]:
X_kag = combined_df.query('train == 0').drop(
    ['price', 'train', 'name', 'price_log2', 'mileage_per_year'], axis=1)

In [637]:
X_kag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34686 entries, 0 to 34685
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   bodyType               34686 non-null  int8   
 1   brand                  34686 non-null  int8   
 2   color                  34686 non-null  int8   
 3   engineDisplacement     34686 non-null  float64
 4   enginePower            34686 non-null  float64
 5   fuelType               34686 non-null  int64  
 6   mileage                34686 non-null  float64
 7   model_name             34686 non-null  int16  
 8   numberOfDoors          34686 non-null  float64
 9   productionDate         34686 non-null  float64
 10  sell_id                34686 non-null  int64  
 11  vehicleTransmission    34686 non-null  int64  
 12  age                    34686 non-null  float64
 13  owners                 34686 non-null  float64
 14  transmission           34686 non-null  int64  
 15  ra

In [638]:
X_kag.enginePower = X_kag.enginePower.apply(
    transf_engineDisplacement_to_float)

In [640]:
predict_submission = np.exp(sr_log.predict(X_kag))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_final.csv', index=False)
sample_submission.head(10)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.1s finished


,sell_id,price
0,1100575026,4.705177e+05
1,1100549428,6.537377e+05
2,1100658222,5.457857e+05
3,1100937408,4.745403e+05
4,1101037972,4.801561e+05
5,1100912634,4.365127e+05
6,1101228730,3.964162e+05
7,1100165896,3.178476e+05
8,1100768262,1.006403e+06
9,1101218501,5.029558e+05
